In [1]:
from pathlib import Path
import json

# load COCO metadata
filename = r"C:\BWSI\Week3\captions_train2014.json"
with Path(filename).open() as f:
    coco_data = json.load(f)

In [2]:
from gensim.models import KeyedVectors
filename = r"C:\BWSI\Week3\glove.6B.200d.txt.w2v"
glove = KeyedVectors.load_word2vec_format(filename, binary=False)

C:\Users\Hello\anaconda3\envs\week3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import re, string

punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))

def strip_punc(corpus):
    return punc_regex.sub('', corpus)

In [27]:
from collections import Counter
import numpy as np
import mygrad as mg

embeddings = {}
word_count = Counter()

for caption_info in coco_data["annotations"]:
    caption = caption_info["caption"]
    caption = strip_punc(caption).lower().split()
    word_count.update(Counter(set(caption)))

for caption_info in coco_data["annotations"]:
    caption = caption_info["caption"]
    caption_id = caption_info["id"]
    caption = strip_punc(caption).lower().split()
    w_caption = np.zeros(200)
    for word in caption:
        idf = np.log10(len(coco_data["annotations"]) / word_count[word])
        if word not in glove:
            w_word = np.zeros(200)
        else:
            w_word = glove[word]
        w_caption += idf * w_word
    w_caption = w_caption / mg.sqrt((w_caption ** 2).sum(keepdims=True))
    embeddings[caption_id] = w_caption.data

In [28]:
import pickle
with open("./embedding.pkl", mode="wb") as emb:
            pickle.dump(embeddings, emb)

In [26]:
embeddings[48]

Tensor([ 0.07061214, -0.02274254, -0.00940832, -0.04032666, -0.03778504,
        -0.00521474, -0.04675333,  0.03444963,  0.02304072, -0.02254693,
         0.01115278,  0.04039144,  0.01566701,  0.02873464,  0.06777503,
         0.03655903, -0.04502166,  0.04222868,  0.06746073, -0.05405859,
         0.03160565,  0.49798158,  0.08423478, -0.05645183,  0.06801857,
        -0.14707932, -0.01028187, -0.07605441, -0.07318673, -0.05556216,
         0.02297116, -0.02974771,  0.06205539,  0.03704237, -0.03858265,
        -0.0183256 , -0.04962339, -0.07829404,  0.07678513,  0.01260141,
         0.01075453,  0.06749363, -0.00623134,  0.06552903, -0.02075985,
        -0.01964161,  0.14423961, -0.04593186, -0.00131347,  0.07078433,
         0.01679406,  0.06363841, -0.0230912 ,  0.08268034,  0.01778664,
        -0.01112581, -0.01539579, -0.06673906, -0.08046044, -0.01877393,
         0.005516  ,  0.03242495, -0.06550016,  0.0571154 , -0.03657656,
         0.05755676, -0.02064952, -0.05495304, -0.0

In [29]:
def create_embeddings(coco_data, glove):
    embeddings = {}
    word_count = Counter()

    for caption_info in coco_data["annotations"]:
        caption = caption_info["caption"]
        caption = strip_punc(caption).lower().split()
        word_count.update(Counter(set(caption)))

    for caption_info in coco_data["annotations"]:
        caption = caption_info["caption"]
        caption_id = caption_info["id"]
        caption = strip_punc(caption).lower().split()
        w_caption = np.zeros(200)
        for word in caption:
            idf = np.log10(len(coco_data["annotations"]) / word_count[word])
            if word not in glove:
                w_word = np.zeros(200)
            else:
                w_word = glove[word]
            w_caption += idf * w_word
        w_caption = w_caption / mg.sqrt((w_caption ** 2).sum(keepdims=True))
        embeddings[caption_id] = w_caption.data
    return embeddings

In [30]:
def create_embedding(query, coco_data, glove):
    
    query = strip_punc(query).lower().split()
    # word_count.update(Counter(set(caption)))
    word_count = Counter()

    for caption_info in coco_data["annotations"]:
        caption = caption_info["caption"]
        caption = strip_punc(caption).lower().split()
        word_count.update(Counter(set(caption)))
        
    w_query = np.zeros(200)
    for word in query:
        idf = np.log10(len(coco_data["annotations"]) / word_count[word])
        if word not in glove:
            w_word = np.zeros(200)
        else:
            w_word = glove[word]
        w_query += idf * w_word
    w_query = w_query / mg.sqrt((w_query ** 2).sum(keepdims=True))
    embedding = w_query.data
    return w_query